## Część 2

In [48]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="secret.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()
query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1

,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,RO,2020-06-04,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
1,RO,2021-10-30,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
2,RO,2021-02-14,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
3,RO,2021-05-06,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
4,RO,2020-12-28,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
5,RO,2020-11-04,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
6,RO,2021-03-11,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
7,RO,2021-05-11,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
8,RO,2021-11-03,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)
9,RO,2020-08-09,ChIJw3aJlSb_sUARlLEEqJJP74Q,Q218,country/ROU,RO,Romania,RO,ROU,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(25 46)


## Część 3

3.1. Sprawdź, ile jest zapisanych wierszy z danymi. 

In [49]:
query = ("""
select count(*) as liczba_wierszy from bigquery-public-data.covid19_open_data.covid19_open_data;
""")

query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_wierszy
0,22756333


 3.2. Sprawdź, ile krajów jest uwzględnionych w danych. 



In [50]:
query = ("""
select count(*) as liczba_krajow FROM (SELECT DISTINCT country_name FROM bigquery-public-data.covid19_open_data.covid19_open_data);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,liczba_krajow
0,246


 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.



Dzienne informacje dla krajów raportowane są z podziałem na części opisywane przez aggregation_level.
Dane odnośnie osób hospitalizowanych, zaszczepionych, zmarłych i testowanych raportowane są oddzielnie dla poszczególnych przedziałów wiekowych oraz z podziałem na płeć. 
Dane zawierają również podstawowe informacje o poszczególnych krajach, takie jak populacja, powierzchnia, wilgotność czy przewidywana długość życia.

 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.  



In [51]:
print(df1['population'].dtype)
print(df1['nurses_per_1000'].dtype)
print(df1['new_tested'].dtype)

Int64
float64
object


 3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.



In [52]:
query = ("""
select min(date) as poczatek_przedzialu, max(date) as koniec_przedzialu FROM bigquery-public-data.covid19_open_data.covid19_open_data;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df
# co to znaczy nowych?

,poczatek_przedzialu,koniec_przedzialu
0,2020-01-01,2022-09-17


In [53]:
query = ("""
select min(date) as poczatek_przedzialu_zachorowan, max(date) as koniec_przedzialu_zachorowan FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed is not null and new_confirmed <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zachorowan,koniec_przedzialu_zachorowan
0,2020-01-01,2022-09-15


In [54]:
query = ("""
select min(date) as poczatek_przedzialu_smierci, max(date) as koniec_przedzialu_smierci FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_deceased is not null and new_deceased <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_smierci,koniec_przedzialu_smierci
0,2020-01-02,2022-09-15


In [55]:
query = ("""
select min(date) as poczatek_przedzialu_zaszczepionych, max(date) as koniec_przedzialu_zaszczepionych FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_persons_vaccinated is not null and new_persons_vaccinated <> 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,poczatek_przedzialu_zaszczepionych,koniec_przedzialu_zaszczepionych
0,2020-12-07,2022-09-14


 3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 



In [56]:
# Sprawdzenie czy wystepuja wartosci ujemne
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where new_confirmed < 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,102199


In [57]:
#Sprawdzenie czy wystepuja duplikaty
query = ("""
select COUNT(*), location_key, date FROM bigquery-public-data.covid19_open_data.covid19_open_data group by location_key, date having count(*) > 1;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_,location_key,date


In [58]:
# Sprawdzenie czy kluczowe kolumny zawieraja nulle
query = ("""
select COUNT(*) FROM bigquery-public-data.covid19_open_data.covid19_open_data where date is null or country_name is null or location_key is null or iso_3166_1_alpha_3 is null;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,f0_
0,991


In [59]:
# Sprawdzenie z jakich krajow posiadamy najwiecej danych
query = ("""
select country_name, count(*) as count from bigquery-public-data.covid19_open_data.covid19_open_data group by country_name order by count desc limit 10;
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,country_name,count
0,Brazil,5565456
1,United States of America,3255435
2,Mexico,2475518
3,Peru,1883891
4,Israel,1472626
5,Spain,1385418
6,Colombia,1145596
7,India,747214
8,Indonesia,544059
9,Argentina,540095


In [60]:
# sprawdzenie czy w kolumnach cumulative wystepuja wartosci mniejsze niz w kolumnach new
query = ("""
select distinct location_key, country_name from bigquery-public-data.covid19_open_data.covid19_open_data where (new_confirmed > cumulative_confirmed or new_deceased > cumulative_deceased or cast(new_tested as int) > cumulative_tested or new_persons_vaccinated > cumulative_persons_vaccinated or new_persons_fully_vaccinated > cumulative_persons_fully_vaccinated or new_vaccine_doses_administered > cumulative_vaccine_doses_administered);
""")
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df

,location_key,country_name
0,MY,Malaysia
1,PL_14,Poland
2,PL_16,Poland
3,PL_22,Poland
4,PL_26,Poland
5,CL,Chile
6,AT,Austria
7,PL_12,Poland
8,PL_20,Poland
9,PL_08,Poland


## Część 4

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.



In [61]:
query = ("""
SELECT distinct
    country_code,
    date,  
    country_name, 
    iso_3166_1_alpha_2, #imo useless
    iso_3166_1_alpha_3, 
    population, 
    population_male, 
    population_female, 
    population_density, 
    human_development_index, 
    gdp_usd, 
    gdp_per_capita_usd, 
    latitude, 
    longitude, 
    area_sq_km, 
    nurses_per_1000, 
    physicians_per_1000,
    health_expenditure_usd, 
    out_of_pocket_health_expenditure_usd 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    country_code IS NOT NULL AND 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    iso_3166_1_alpha_2 IS NOT NULL AND 
    iso_3166_1_alpha_3 IS NOT NULL AND 
    population IS NOT NULL AND 
    population_male IS NOT NULL AND 
    population_female IS NOT NULL AND 
    population_density IS NOT NULL AND 
    human_development_index IS NOT NULL AND 
    gdp_usd IS NOT NULL AND 
    gdp_per_capita_usd IS NOT NULL AND 
    latitude IS NOT NULL AND 
    longitude IS NOT NULL AND 
    area_sq_km IS NOT NULL AND 
    nurses_per_1000 IS NOT NULL AND 
    physicians_per_1000 IS NOT NULL AND 
    health_expenditure_usd IS NOT NULL AND 
    out_of_pocket_health_expenditure_usd IS NOT NULL
    AND aggregation_level = 0;
""")
query_job = client.query(query)
query_result = query_job.result()
df1 = query_result.to_dataframe()
df1.to_csv("countries_data.csv", index=False)
df1

,country_code,date,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,population_density,human_development_index,gdp_usd,gdp_per_capita_usd,latitude,longitude,area_sq_km,nurses_per_1000,physicians_per_1000,health_expenditure_usd,out_of_pocket_health_expenditure_usd
0,CD,2020-03-23,Democratic Republic of the Congo,CD,COD,89561404,44710172,44851232,39.506,0.457,47319624204,545,-2.880000,23.6560,2344860,1.1101,0.074,19.431646,7.796894
1,CD,2021-12-15,Democratic Republic of the Congo,CD,COD,89561404,44710172,44851232,39.506,0.457,47319624204,545,-2.880000,23.6560,2344860,1.1101,0.074,19.431646,7.796894
2,CD,2021-03-15,Democratic Republic of the Congo,CD,COD,89561404,44710172,44851232,39.506,0.457,47319624204,545,-2.880000,23.6560,2344860,1.1101,0.074,19.431646,7.796894
3,CD,2022-01-22,Democratic Republic of the Congo,CD,COD,89561404,44710172,44851232,39.506,0.457,47319624204,545,-2.880000,23.6560,2344860,1.1101,0.074,19.431646,7.796894
4,CD,2021-06-04,Democratic Republic of the Congo,CD,COD,89561404,44710172,44851232,39.506,0.457,47319624204,545,-2.880000,23.6560,2344860,1.1101,0.074,19.431646,7.796894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151618,US,2021-01-14,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,21374418877706,65118,39.828175,-98.5795,9831510,14.5480,2.612,10246.138672,1126.349976
151619,US,2021-10-26,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,21374418877706,65118,39.828175,-98.5795,9831510,14.5480,2.612,10246.138672,1126.349976
151620,US,2020-02-26,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,21374418877706,65118,39.828175,-98.5795,9831510,14.5480,2.612,10246.138672,1126.349976
151621,US,2021-04-02,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,21374418877706,65118,39.828175,-98.5795,9831510,14.5480,2.612,10246.138672,1126.349976


 4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.



In [62]:
query = ("""select distinct date, country_name, new_confirmed, cumulative_confirmed, cast(new_tested as int) as new_tested,  cumulative_tested from bigquery-public-data.covid19_open_data.covid19_open_data WHERE
  date IS NOT NULL
  AND country_name IS NOT NULL
  AND new_confirmed IS NOT NULL
  AND cumulative_confirmed IS NOT NULL
  AND new_tested IS NOT NULL
  AND cumulative_tested IS NOT NULL
  AND new_confirmed <= cumulative_confirmed
  and cast(new_tested as int) <= cumulative_tested
  AND new_confirmed >= 0
  AND cast(new_tested as int) >= 0
  AND cumulative_confirmed >= 0
  AND cumulative_tested >= 0
  AND aggregation_level = 0
  """)
query_job = client.query(query)
query_result = query_job.result()
df2 = query_result.to_dataframe()
df2.to_csv("covid_cases_stats.csv", index=False)
df2

,date,country_name,new_confirmed,cumulative_confirmed,new_tested,cumulative_tested
0,2021-07-27,Madagascar,30,42715,511,226580
1,2020-12-09,Mongolia,5,898,21470,366851
2,2021-02-25,Mongolia,35,2866,9927,1759786
3,2022-03-03,Saint Vincent and the Grenadines,4,8320,3,98329
4,2020-07-25,Belarus,130,67132,9735,1251359
...,...,...,...,...,...,...
73510,2021-03-05,Zimbabwe,12,36260,1600,392499
73511,2021-03-02,Zimbabwe,31,36179,1903,387311
73512,2021-04-25,Zimbabwe,16,38102,1495,484446
73513,2021-01-30,Zimbabwe,117,33388,1762,318260


 4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.



In [63]:
query = ("""
SELECT 
    date,  
    country_name,  
    new_deceased, 
    cumulative_deceased 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    new_deceased IS NOT NULL AND 
    cumulative_deceased IS NOT NULL AND
    new_deceased <= cumulative_deceased AND
    new_deceased >= 0
    AND cumulative_deceased >= 0
    AND aggregation_level = 0

    ;

""")
query_job = client.query(query)
query_result = query_job.result()
df3 = query_result.to_dataframe()
df3.to_csv("covid_mortality.csv", index=False)
df3

,date,country_name,new_deceased,cumulative_deceased
0,2022-09-11,Andorra,0,155
1,2020-02-24,Andorra,0,0
2,2021-04-28,Andorra,0,125
3,2022-04-22,Andorra,0,153
4,2022-09-01,Andorra,0,154
...,...,...,...,...
227569,2021-12-04,Zimbabwe,1,4710
227570,2021-10-25,Zimbabwe,1,4674
227571,2022-01-09,Zimbabwe,20,5180
227572,2020-02-26,Estonia,0,0


 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.



In [64]:
query = ("""
SELECT 
    date,  
    country_name, 
    new_persons_vaccinated, 
    cumulative_persons_vaccinated, 
    new_persons_fully_vaccinated, 
    cumulative_persons_fully_vaccinated, 
    new_vaccine_doses_administered, 
    cumulative_vaccine_doses_administered 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    new_persons_vaccinated IS NOT NULL AND 
    cumulative_persons_vaccinated IS NOT NULL AND 
    new_persons_fully_vaccinated IS NOT NULL AND 
    cumulative_persons_fully_vaccinated IS NOT NULL AND 
    new_vaccine_doses_administered IS NOT NULL AND 
    cumulative_vaccine_doses_administered IS NOT NULL AND
    new_persons_vaccinated <= cumulative_persons_vaccinated AND
    new_persons_fully_vaccinated <= cumulative_persons_fully_vaccinated AND
    new_vaccine_doses_administered <= cumulative_vaccine_doses_administered AND
    new_persons_vaccinated >= 0 AND
    new_persons_fully_vaccinated >= 0 AND
    new_vaccine_doses_administered >= 0 AND
    cumulative_persons_vaccinated >= 0 AND
    cumulative_persons_fully_vaccinated >= 0 AND
    cumulative_vaccine_doses_administered >= 0
    AND aggregation_level = 0

    ;

""")
query_job = client.query(query)
query_result = query_job.result()
df4 = query_result.to_dataframe()
df4.to_csv("covid_vaccination.csv", index=False)
df4

,date,country_name,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered
0,2021-05-03,Andorra,360,24182,18,4699,378,28881
1,2021-04-05,Andorra,2683,9781,3219,4484,2829,15269
2,2021-11-10,United Arab Emirates,10013,9771891,10001,8772943,26974,21404200
3,2022-09-12,Afghanistan,3885,10827184,9432,10105500,9447,11734635
4,2022-03-02,Anguilla,69,10439,18,9549,212,22377
...,...,...,...,...,...,...,...,...
47216,2021-11-30,Zimbabwe,13363,3795401,12076,2816541,25439,6611942
47217,2022-05-23,Zimbabwe,2669,6225138,5513,4480054,9540,11527420
47218,2021-05-08,Zimbabwe,8852,509272,8224,148544,17076,657816
47219,2022-03-15,Zimbabwe,4345,4405342,2840,3434337,10885,8013538


 4.5. Zdefiniuj własny dodatkowy przypadek. Chcemy dowiedzieć się ile jest pielęgniarek i lekarzy w poszczególnych krajach.



In [65]:
query = ("""
SELECT 
    date,  
    country_name, 
    nurses_per_1000,
    physicians_per_1000
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE 
    date IS NOT NULL AND  
    country_name IS NOT NULL AND 
    nurses_per_1000 >= 0 AND
    physicians_per_1000 >= 0 AND
    aggregation_level = 0
    ;
""")
query_job = client.query(query)
query_result = query_job.result()
df5 = query_result.to_dataframe()
df5.to_csv("covid_vaccination.csv", index=False)
df5

,date,country_name,nurses_per_1000,physicians_per_1000
0,2020-10-20,Afghanistan,0.1755,0.2782
1,2020-12-23,Afghanistan,0.1755,0.2782
2,2021-11-06,Afghanistan,0.1755,0.2782
3,2020-10-27,Afghanistan,0.1755,0.2782
4,2021-08-03,Afghanistan,0.1755,0.2782
...,...,...,...,...
162519,2021-08-12,Zimbabwe,1.9346,0.2096
162520,2021-06-19,Zimbabwe,1.9346,0.2096
162521,2022-02-12,Zimbabwe,1.9346,0.2096
162522,2021-11-22,Zimbabwe,1.9346,0.2096


## Część 5
Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [66]:
merged_df = df1.merge(df2, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df3, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df4, on=["country_name", "date"], how="inner")
merged_df = merged_df.merge(df5, on=["country_name", "date"], how="inner")
merged_df = merged_df.dropna()
merged_df.to_csv("covid_combined_data.csv", index=False)
merged_df

,country_code,date,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,population_density,human_development_index,...,new_deceased,cumulative_deceased,new_persons_vaccinated,cumulative_persons_vaccinated,new_persons_fully_vaccinated,cumulative_persons_fully_vaccinated,new_vaccine_doses_administered,cumulative_vaccine_doses_administered,nurses_per_1000_y,physicians_per_1000_y
0,MZ,2021-07-19,Mozambique,MZ,MOZ,31255435,15188235,16067200,39.746,0.437,...,20,1158,17513,372085,145639,299251,163152,671336,0.6847,0.0838
1,MZ,2021-08-03,Mozambique,MZ,MOZ,31255435,15188235,16067200,39.746,0.437,...,26,1526,22149,394234,1587,320542,23736,714776,0.6847,0.0838
2,MZ,2021-08-25,Mozambique,MZ,MOZ,31255435,15188235,16067200,39.746,0.437,...,3,1825,727346,1643520,82936,654322,433613,2297842,0.6847,0.0838
3,MZ,2021-06-24,Mozambique,MZ,MOZ,31255435,15188235,16067200,39.746,0.437,...,2,865,21578,341707,21874,96675,43452,438382,0.6847,0.0838
4,MZ,2021-09-14,Mozambique,MZ,MOZ,31255435,15188235,16067200,39.746,0.437,...,3,1898,260210,1903730,978323,1740693,1238533,3644423,0.6847,0.0838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23110,US,2021-06-09,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,...,366,565048,393095,177060408,538153,140980110,1171970,323330247,14.5480,2.6120
23111,US,2021-03-30,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,...,659,525030,2005987,106208785,809255,53423486,3583688,167899966,14.5480,2.6120
23112,US,2021-01-14,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,...,3643,379978,1157564,14296098,136781,2262911,1319739,16888720,14.5480,2.6120
23113,US,2021-10-26,United States of America,US,USA,331002647,163786016,167216631,36.185,0.920,...,2005,708915,295925,220645590,93310,190793100,1569375,425197648,14.5480,2.6120


## Część 6
Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach

In [68]:
world_countries = pd.read_csv("world_countries.csv")
world_countries = world_countries.rename(columns={"CCA3": "iso_3166_1_alpha_3", "Country/Territories": "country_name", "Area (km²)": "area_sq_km", "Density (per km²)": "population_density"})

gdp = pd.read_csv("GDP.csv")
gdp = gdp.rename(columns={"Country Code": "iso_3166_1_alpha_3", "Country Name": "country_name", "Value": "gdp_usd", "Year":"year"})
merged_df["year"] = pd.to_datetime(merged_df["date"]).dt.year
merged_df2 = merged_df.merge(world_countries, on="iso_3166_1_alpha_3", how="inner")
merged_df2 = merged_df2.merge(gdp, on=["iso_3166_1_alpha_3","year"], how="inner")
merged_df2
# nie rozumiem troche, imo bez sensu mergowac to z tymi calymi danymi koncowymi z 5, a w GDP country name to wgl nazwa kraju lub regionu wiec troche xd

,country_code,date,country_name_x,iso_3166_1_alpha_2,iso_3166_1_alpha_3,population,population_male,population_female,population_density_x,human_development_index,...,2000 Population,1990 Population,1980 Population,1970 Population,area_sq_km_y,population_density_y,Growth Rate,World Population Percentage,country_name_y,gdp_usd_y
